## Data Manpulation
In this notebook i will show how to convert the labels to different represntations. As all of you know there is already a pretrained model https://github.com/Kohulan/DECIMER-Image-to-SMILES to Extract SMILES representation from images.

In [ ]:
import numpy as np 
import pandas as pd 
import os
import cv2
import requests

In [ ]:
df = pd.read_csv('../input/bms-molecular-translation/train_labels.csv')
df.head()

In [ ]:
inchi = df['InChI'].iloc[0]
inchi

In [ ]:
## you can send a direct request to get the SMILES representation 
HOST = "http://www.chemspider.com"
Operation = "/InChI.asmx/InChIToInChIKey?inchi="

request = requests.get('{}{}{}'.format(HOST, Operation, inchi))
if request.ok:
    res = str(request.text.replace('<?xml version="1.0" encoding="utf-8"?>\r\n<string xmlns="http://www.chemspider.com/">', '').replace('</string>', '').strip())
else:
    print ("provide a valid inchi!")
print(res)

In [ ]:
!pip install bioservices

In [ ]:
#get information about the Molecular
from bioservices import *
info = UniChem()
cid = info.get_src_compound_ids_from_inchikey(res)[0]

In [ ]:
# Now let us go to SMILES using another service for FUN 
def get_smiles_from_inchikey(inchikey):
    request = requests.get(f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/{inchikey}/property/CanonicalSMILES/JSON').json()
    return request['PropertyTable']['Properties'][0]['CanonicalSMILES']

In [ ]:
smiles_pres=get_smiles_from_inchikey(res)
print(smiles_pres)

In [ ]:
# new library ))
!pip install -U chembl_webresource_client

In [ ]:
# go back from InChiKey to In Chi
from chembl_webresource_client.unichem import unichem_client as unichem
ret = unichem.inchiFromKey(res)
print (ret[0]['standardinchi'])
print(inchi)

## Let us Generate a good images

In [ ]:
!wget -c https://repo.continuum.io/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh
!chmod +x Miniconda3-py37_4.8.3-Linux-x86_64.sh
!time bash ./Miniconda3-py37_4.8.3-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge rdkit

import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# ref: https://www.kaggle.com/ihelon/molecular-translation-exploratory-data-analysis 
def convert_image_id_2_path(image_id: str) -> str:
    return "../input/bms-molecular-translation/train/{}/{}/{}/{}.png".format(
        image_id[0], image_id[1], image_id[2], image_id 
    )

In [ ]:
#ref: https://www.kaggle.com/ihelon/molecular-translation-exploratory-data-analysis
def visualize_train_image(image_id, label):
    plt.figure(figsize=(8, 8))
    print(image_id)
    image = cv2.imread(convert_image_id_2_path(image_id))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(image)
    plt.title(f"{label}", fontsize=14)
    plt.axis("off")
    
    plt.show()

In [ ]:
sample_row = df.iloc[0]
visualize_train_image(
        sample_row["image_id"], sample_row["InChI"]
    )

In [ ]:
penicillin_g_smiles = smiles_pres

penicillin_g = Chem.MolFromSmiles(penicillin_g_smiles)

Draw.MolToMPL(penicillin_g, size=(200, 200))

In [ ]:
!pip install py3Dmol

In [ ]:
import py3Dmol

In [ ]:
#Notebook ref: https://www.kaggle.com/maunish/bms-super-cool-eda
def show_3d_models(cid):
    view = py3Dmol.view(width=600, height=1000, query=cid, viewergrid=(2,1), linked=False)
    view.setStyle({'stick': {}}, viewer=(0,0))
    view.setStyle({'sphere': {}}, viewer=(1,0))
    view.setBackgroundColor('#1AD40D', viewer=(0,0))
    view.setBackgroundColor('#1AD40D', viewer=(1,0))
    view.show()

In [ ]:
cid = 'cid:'+cid['src_compound_id']
show_3d_models(cid)

## Note
Using DECIMER directly will give a really bad results the first step is to make image denoisying.
Hope it will be useful